Data Source: https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant

Features consist of hourly average ambient variables

Temperature (T) in the range 1.81°C and 37.11°C,
Ambient Pressure (AP) in the range 992.89-1033.30 milibar,
Relative Humidity (RH) in the range 25.56% to 100.16%
Exhaust Vacuum (V) in teh range 25.36-81.56 cm Hg
Net hourly electrical energy output (EP) 420.26-495.76 MW
The averages are taken from various sensors located around the plant that record the ambient variables every second. The variables are given without normalization.

Dataset Information:

The dataset contains 9568 data points collected from a Combined Cycle Power Plant over 6 years (2006-2011), when the power plant was set to work with full load. Features consist of hourly average ambient variables Temperature (T), Ambient Pressure (AP), Relative Humidity (RH) and Exhaust Vacuum (V) to predict the net hourly electrical energy output (EP) of the plant. 
A combined cycle power plant (CCPP) is composed of gas turbines (GT), steam turbines (ST) and heat recovery steam generators. In a CCPP, the electricity is generated by gas and steam turbines, which are combined in one cycle, and is transferred from one turbine to another. While the Vacuum is colected from and has effect on the Steam Turbine, he other three of the ambient variables effect the GT performance.

In [3]:
!ls -ltr ~/Downloads/datasets

total 345560
drwxr-xr-x 2 training training      4096 Oct 17  2016 ml-20m
-rw-r--r-- 1 training training 127744095 Mar 24 20:20 stocks.csv
drwxrwxrwx 2 training training      4096 Jun 18 21:08 ml-latest-small
-rw-r--r-- 1 training training 150828752 Sep 13 15:33 creditcard-fraud.csv
drwxrwxr-x 2 training training      4096 Sep 13 20:35 mnist
-rw-rw-r-- 1 training training    133638 Sep 13 21:00 credit.csv
-rw-rw-r-- 1 training training  69055807 Sep 13 21:01 imdb-comments.json
-rw-rw-r-- 1 training training      5107 Sep 13 21:01 iris.csv
-rw-rw-r-- 1 training training     57459 Sep 13 21:01 istanbul-stock.csv
-rw-rw-r-- 1 training training       226 Sep 13 21:01 mobile-sales-data.csv
-rw-rw-r-- 1 training training      2436 Sep 13 21:01 startups.csv
drwxrwxr-x 2 training training      4096 Sep 13 21:01 kaggle-house-prices
-rw-rw-r-- 1 training training   4432265 Sep 13 23:51 Bible.txt
drwxrwxr-x 2 training training      4096 Sep 15 05:58 apple.stackexchange.com
-rw-rw-r-- 1 training t

# Load Data

In [4]:
df = spark.read.format("csv").option("header","true")\
.option("inferSchema","true").load("/home/training/Downloads/datasets/Combined_Cycle_Power_Plant.csv")

In [5]:
df.show()

+-----+-----+-------+-----+------+
|   AT|    V|     AP|   RH|    PE|
+-----+-----+-------+-----+------+
|14.96|41.76|1024.07|73.17|463.26|
|25.18|62.96|1020.04|59.08|444.37|
| 5.11| 39.4|1012.16|92.14|488.56|
|20.86|57.32|1010.24|76.64|446.48|
|10.82| 37.5|1009.23|96.62| 473.9|
|26.27|59.44|1012.23|58.77|443.67|
|15.89|43.96|1014.02|75.24|467.35|
| 9.48|44.71|1019.12|66.43|478.42|
|14.64| 45.0|1021.78|41.25|475.98|
|11.74|43.56|1015.14|70.72| 477.5|
|17.99|43.72|1008.64|75.04|453.02|
|20.14|46.93|1014.66|64.22|453.99|
|24.34| 73.5|1011.31|84.15|440.29|
|25.71|58.59|1012.77|61.83|451.28|
|26.19|69.34|1009.48|87.59|433.99|
|21.42|43.79|1015.76|43.08|462.19|
|18.21| 45.0|1022.86|48.84|467.54|
|11.04|41.74| 1022.6|77.51| 477.2|
|14.45|52.75|1023.97|63.59|459.85|
|13.97|38.47|1015.15|55.28| 464.3|
+-----+-----+-------+-----+------+
only showing top 20 rows



In [6]:
df.cache()

DataFrame[AT: double, V: double, AP: double, RH: double, PE: double]

# Convert Spark Dataframe to Pandas Dataframe

In [9]:
df.toPandas().head() #Do not run on large dataset

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


## Verctorize the features

In [10]:
from pyspark.ml.feature import *

In [53]:
vectorizer = VectorAssembler()
vectorizer.setInputCols(["AT", "V", "AP", "RH"])
vectorizer.setOutputCol("features")

df_vect = vectorizer.transform(df)
df_vect.show()

+-----+-----+-------+-----+------+--------------------+
|   AT|    V|     AP|   RH|    PE|            features|
+-----+-----+-------+-----+------+--------------------+
|14.96|41.76|1024.07|73.17|463.26|[14.96,41.76,1024...|
|25.18|62.96|1020.04|59.08|444.37|[25.18,62.96,1020...|
| 5.11| 39.4|1012.16|92.14|488.56|[5.11,39.4,1012.1...|
|20.86|57.32|1010.24|76.64|446.48|[20.86,57.32,1010...|
|10.82| 37.5|1009.23|96.62| 473.9|[10.82,37.5,1009....|
|26.27|59.44|1012.23|58.77|443.67|[26.27,59.44,1012...|
|15.89|43.96|1014.02|75.24|467.35|[15.89,43.96,1014...|
| 9.48|44.71|1019.12|66.43|478.42|[9.48,44.71,1019....|
|14.64| 45.0|1021.78|41.25|475.98|[14.64,45.0,1021....|
|11.74|43.56|1015.14|70.72| 477.5|[11.74,43.56,1015...|
|17.99|43.72|1008.64|75.04|453.02|[17.99,43.72,1008...|
|20.14|46.93|1014.66|64.22|453.99|[20.14,46.93,1014...|
|24.34| 73.5|1011.31|84.15|440.29|[24.34,73.5,1011....|
|25.71|58.59|1012.77|61.83|451.28|[25.71,58.59,1012...|
|26.19|69.34|1009.48|87.59|433.99|[26.19,69.34,1

In [16]:
print(vectorizer.explainParams())

inputCols: input column names. (current: ['AT', 'V', 'AP', 'RH'])
outputCol: output column name. (default: VectorAssembler_406dae73f80cb9440a18__output)


## Fit Linear Regression Model

In [21]:
from pyspark.ml.regression import LinearRegression

In [24]:
lr = LinearRegression()
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0)
solver: the solver algorithm for optimization. If this is not set or empty, default value is 'auto'. (default: auto)
standardization: whether to standardize the training features before fitting the model. (default: True)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)


In [54]:
lr.setLabelCol("PE")
lr.setFeaturesCol("features")
model = lr.fit(df_vect)

In [55]:
type(model)

pyspark.ml.regression.LinearRegressionModel

### View model summary

In [56]:
print("R2:", model.summary.r2)
print("Intercept: ", model.intercept, "Coefficients", model.coefficients)

R2: 0.9286835997167648
Intercept:  454.5637357984046 Coefficients [-1.97731604346,-0.234028456499,0.0621277600987,-0.158016554398]


### Predict

In [61]:
df_pred = model.transform(df_vect)
df_pred.show()

+-----+-----+-------+-----+------+--------------------+------------------+
|   AT|    V|     AP|   RH|    PE|            features|        prediction|
+-----+-----+-------+-----+------+--------------------+------------------+
|14.96|41.76|1024.07|73.17|463.26|[14.96,41.76,1024...| 467.2711634437306|
|25.18|62.96|1020.04|59.08|444.37|[25.18,62.96,1020...|444.07766858004396|
| 5.11| 39.4|1012.16|92.14|488.56|[5.11,39.4,1012.1...|483.56251796945776|
|20.86|57.32|1010.24|76.64|446.48|[20.86,57.32,1010...| 450.5559716382537|
|10.82| 37.5|1009.23|96.62| 473.9|[10.82,37.5,1009....| 471.8267489278455|
|26.27|59.44|1012.23|58.77|443.67|[26.27,59.44,1012...| 442.3099415850402|
|15.89|43.96|1014.02|75.24|467.35|[15.89,43.96,1014...|463.96591866241715|
| 9.48|44.71|1019.12|66.43|478.42|[9.48,44.71,1019....| 478.1739705793852|
|14.64| 45.0|1021.78|41.25|475.98|[14.64,45.0,1021....|472.04726822434776|
|11.74|43.56|1015.14|70.72| 477.5|[11.74,43.56,1015...| 473.0492095425741|
|17.99|43.72|1008.64|75.0

### Evaluate

In [52]:
from pyspark.ml.evaluation import RegressionEvaluator

In [63]:
evaluator = RegressionEvaluator()
print(evaluator.explainParams())

labelCol: label column name. (default: label)
metricName: metric name in evaluation - one of:
                       rmse - root mean squared error (default)
                       mse - mean squared error
                       r2 - r^2 metric
                       mae - mean absolute error. (default: rmse)
predictionCol: prediction column name. (default: prediction)


In [65]:
evaluator = RegressionEvaluator(labelCol = "PE", predictionCol = "prediction", metricName = "rmse")
evaluator.evaluate(df_pred)

4.557525128298466

## Build a pipeline

In [99]:
from pyspark.ml.pipeline import Pipeline, PipelineModel

In [68]:
pipeline = Pipeline()
print(pipeline.explainParams())
pipeline.setStages([vectorizer, lr])
pipelineModel = pipeline.fit(df)

stages: a list of pipeline stages (undefined)


In [44]:
pipeline.getStages()

[VectorAssembler_4fbf8efe8908beb31164, LinearRegression_4bf49d4e37c6cf84ec09]

In [75]:
lr_model = pipelineModel.stages[1]
lr_model .coefficients

DenseVector([-1.9773, -0.234, 0.0621, -0.158])

In [79]:
pipelineModel.transform(df).show()

+-----+-----+-------+-----+------+--------------------+------------------+
|   AT|    V|     AP|   RH|    PE|            features|        prediction|
+-----+-----+-------+-----+------+--------------------+------------------+
|14.96|41.76|1024.07|73.17|463.26|[14.96,41.76,1024...| 467.2711634437306|
|25.18|62.96|1020.04|59.08|444.37|[25.18,62.96,1020...|444.07766858004396|
| 5.11| 39.4|1012.16|92.14|488.56|[5.11,39.4,1012.1...|483.56251796945776|
|20.86|57.32|1010.24|76.64|446.48|[20.86,57.32,1010...| 450.5559716382537|
|10.82| 37.5|1009.23|96.62| 473.9|[10.82,37.5,1009....| 471.8267489278455|
|26.27|59.44|1012.23|58.77|443.67|[26.27,59.44,1012...| 442.3099415850402|
|15.89|43.96|1014.02|75.24|467.35|[15.89,43.96,1014...|463.96591866241715|
| 9.48|44.71|1019.12|66.43|478.42|[9.48,44.71,1019....| 478.1739705793852|
|14.64| 45.0|1021.78|41.25|475.98|[14.64,45.0,1021....|472.04726822434776|
|11.74|43.56|1015.14|70.72| 477.5|[11.74,43.56,1015...| 473.0492095425741|
|17.99|43.72|1008.64|75.0

In [81]:
evaluator.evaluate(pipelineModel.transform(df))

4.557525128298466

## Save the pipeline to disk to persist the model

In [98]:
pipelineModel.save("lr-pipeline")

### Load the persisted model from the disk

In [106]:
saved_model = PipelineModel.load("lr-pipeline")
saved_model.stages[1].coefficients

DenseVector([-1.9773, -0.234, 0.0621, -0.158])

In [108]:
saved_model.transform(df).show()

+-----+-----+-------+-----+------+--------------------+------------------+
|   AT|    V|     AP|   RH|    PE|            features|        prediction|
+-----+-----+-------+-----+------+--------------------+------------------+
|14.96|41.76|1024.07|73.17|463.26|[14.96,41.76,1024...| 467.2711634437306|
|25.18|62.96|1020.04|59.08|444.37|[25.18,62.96,1020...|444.07766858004396|
| 5.11| 39.4|1012.16|92.14|488.56|[5.11,39.4,1012.1...|483.56251796945776|
|20.86|57.32|1010.24|76.64|446.48|[20.86,57.32,1010...| 450.5559716382537|
|10.82| 37.5|1009.23|96.62| 473.9|[10.82,37.5,1009....| 471.8267489278455|
|26.27|59.44|1012.23|58.77|443.67|[26.27,59.44,1012...| 442.3099415850402|
|15.89|43.96|1014.02|75.24|467.35|[15.89,43.96,1014...|463.96591866241715|
| 9.48|44.71|1019.12|66.43|478.42|[9.48,44.71,1019....| 478.1739705793852|
|14.64| 45.0|1021.78|41.25|475.98|[14.64,45.0,1021....|472.04726822434776|
|11.74|43.56|1015.14|70.72| 477.5|[11.74,43.56,1015...| 473.0492095425741|
|17.99|43.72|1008.64|75.0

In [110]:
df_train, df_test = df.randomSplit(weights=[0.7, 0.3], seed = 200)

In [112]:
pipelineModel = pipeline.fit(df_train)
evaluator.evaluate(pipelineModel.transform(df_test))

4.563138184940591

# Tune the model

In [115]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [119]:
paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# In this case the estimator is simply the linear regression.
# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator,
                           trainRatio=0.8)

tuned_model = tvs.fit(vectorizer.transform(df_train))

In [138]:
tuned_model.bestModel, tuned_model.validationMetrics

(LinearRegression_4bf49d4e37c6cf84ec09,
 [5.039347932903683,
  5.636217630762049,
  5.107247882789134,
  4.547646522666319,
  4.545696090598205,
  4.545352062329218,
  5.038546732463139,
  5.49137539732243,
  5.628475999107409,
  4.545755899303294,
  4.5455883701354445,
  4.545438239526134])

In [125]:
df_test_pred = tuned_model.transform(vectorizer.transform(df_test))
df_test_pred.show()

+----+-----+-------+-----+------+--------------------+------------------+
|  AT|    V|     AP|   RH|    PE|            features|        prediction|
+----+-----+-------+-----+------+--------------------+------------------+
|1.81|39.42|1026.92|76.97|490.55|[1.81,39.42,1026....| 492.8349077427656|
|1.81|39.42|1026.92|76.97|490.55|[1.81,39.42,1026....| 492.8349077427656|
|1.81|39.42|1026.92|76.97|490.55|[1.81,39.42,1026....| 492.8349077427656|
|1.81|39.42|1026.92|76.97|490.55|[1.81,39.42,1026....| 492.8349077427656|
|2.34|39.42|1028.47|69.68|490.34|[2.34,39.42,1028....|492.94214880761615|
|2.58|39.42|1028.68|69.03|488.69|[2.58,39.42,1028....|492.58206703142434|
|2.58|39.42|1028.68|69.03|488.69|[2.58,39.42,1028....|492.58206703142434|
| 2.8|39.64|1011.01|82.96|482.66|[2.8,39.64,1011.0...| 489.0569119382538|
| 2.8|39.64|1011.01|82.96|482.66|[2.8,39.64,1011.0...| 489.0569119382538|
|3.21|38.44| 1016.9|86.34|491.35|[3.21,38.44,1016....| 488.4152185498315|
|3.21|38.44|1017.11|84.86|492.93|[3.21

In [126]:
evaluator.evaluate(df_test_pred)

4.5683171800255895